# Setting up the environment (API keys + Jupyter kernel)

Målet: få et ryddig, GitHub-vennlig oppsett der **hemmeligheter ikke havner i repoet**, og der Jupyter bruker riktig Python-miljø (kernel).


Dette kapittelet viser hvordan du lagrer API-nøkler trygt og får Jupyter til å bruke riktig miljø.

## 0) Viktig sikkerhet (GitHub)

- **Aldri** hardcode API keys i notebook eller `.py`.
- **Aldri** commit `.env` til GitHub.
- Bruk heller miljøvariabler + `.env` lokalt.

Hvis du mistenker at en nøkkel er lekket:

- revoke/roter nøkkelen i leverandørens dashboard
- lag ny nøkkel

## 1) Hva er en environment variable?

En miljøvariabel er en **key/value** som OS-et og programmer kan lese.

- Eksempler: `PATH`, `HOME`
- I AI-kode: `OPENAI_API_KEY`, `HF_TOKEN`, `WANDB_API_KEY`, osv.

Fordeler:

- du kan bytte nøkkel uten å endre kode
- du unngår å “lekke” nøkkelen når du deler repo

## 2) Sørg for riktig kernel i Jupyter

Du må kjøre notebooken med samme Python-miljø som har pakkene dine installert.

- I Jupyter: **Kernel → Change Kernel** og velg miljøet ditt (f.eks. `langchain_env`).

> Hvis du ikke ser miljøet ditt som kernel, se “Ekstra: lag kernel” lenger nede i denne notebooken.

## 3) Lag en `.env` fil

I samme mappe som notebooken (eller i prosjekt-roten), lag en fil som heter `.env`:

```env
OPENAI_API_KEY="sk-..."
```

Tips:

- bruk anførselstegn hvis nøkkelen kan inneholde spesialtegn
- ha **én variabel per linje**

## 4) Last `.env` inn i notebooken

Det finnes to vanlige metoder:

- **A)** IPython magic (enkelt i notebook)
- **B)** `python-dotenv` (mer “vanlig Python”, funker i `.py` også)

Nedenfor viser vi begge.
 

In [ ]:
%load_ext dotenv

## 4A) Metode A — IPython magic (`%dotenv`)

Denne metoden er fin i Jupyter.

1) Last extension
2) Kjør `%dotenv` for å lese `.env`

> Hvis dette ikke funker, mangler du ofte pakken `ipython-dotenv` (eller tilsvarende). Da kan du bruke Metode B.
 

In [ ]:
%dotenv

## 4B) Metode B — `python-dotenv` (anbefalt)

Denne metoden funker både i Jupyter og i vanlige Python-scripts.

1) Installer (én gang i miljøet):

```bash
python -m pip install python-dotenv
```

2) Bruk i kode:

```python
import os
from dotenv import load_dotenv

load_dotenv()  # leser .env i current directory/prosjektet
api_key = os.getenv("OPENAI_API_KEY")
```

Fordeler:

- funker overalt (ikke bare notebooks)
- lettere å debugge

> Etter dette er du klar til å lage første chatbot – men nå har du gjort det på en måte som er trygg for GitHub.


In [ ]:
# Verifiser at env-variabelen er tilgjengelig (uten å lekke nøkkelen)
import os

key = os.getenv("OPENAI_API_KEY")

if not key:
    raise RuntimeError(
        "OPENAI_API_KEY er ikke satt. Sjekk at du har .env i riktig mappe og at du har kjørt %dotenv eller load_dotenv()."
    )

# Trygt: print bare lengde og prefix
print("OPENAI_API_KEY loaded:", True)
print("Length:", len(key))
print("Prefix:", key[:3] + "...")

In [ ]:
# Bonus: sjekk hvilket miljø/kernel du faktisk kjører
import sys
import platform

print("Python:", sys.version)
print("Executable:", sys.executable)
print("Platform:", platform.platform())

## 6) Ekstra: lag et eget Python-miljø (anbefalt)

På Mac er det vanlig å bruke enten `venv` (enkelt) eller `pyenv` (flere Python-versjoner).

### A) `venv` (enkelt og bra nok)

Kjør dette i prosjektmappa:

```bash
python3 -m venv .venv
source .venv/bin/activate
python -m pip install --upgrade pip
```

Når du ser `(.venv)` i terminalen, er miljøet aktivt.

### B) `pyenv` (hvis du vil styre Python-versjon)

Du bruker allerede `pyenv` på maskinen din. Poenget er å ha en fast Python-versjon per prosjekt.


## 7) Ekstra: få miljøet ditt som Jupyter kernel

Hvis du har installert pakker i `.venv`, men Jupyter ikke bruker dem, må du registrere miljøet som en kernel.

```bash
python -m pip install ipykernel
python -m ipykernel install --user --name langchain_env --display-name "langchain_env"
```

Deretter i notebook:

- **Kernel → Change Kernel → `langchain_env`**


## 8) Ekstra: `requirements.txt` (så andre kan kjøre repoet ditt)

For GitHub er det veldig fint å kunne installere dependencies med én kommando.

### A) Lag `requirements.txt`

Typisk:

```bash
python -m pip freeze > requirements.txt
```

### B) Installer fra `requirements.txt`

```bash
python -m pip install -r requirements.txt
```

> Tips: i større prosjekter kan du ha `requirements-dev.txt` også (formatting, linting, etc.).


## 9) GitHub hygiene: `.gitignore` + `.env.example`

For at repoet ditt skal være trygt og “clean”:

### A) `.gitignore`

Sørg for at minst dette ikke blir committed:

```gitignore
# secrets
.env

# python
__pycache__/
*.pyc

# venv
.venv/

# jupyter
.ipynb_checkpoints/
```

### B) `.env.example`

Lag en fil du **kan** committe (uten ekte nøkkel):

```env
OPENAI_API_KEY=""
```

Da kan andre kopiere den til `.env` lokalt.


## 10) Vanlige problemer (og raske fixes)

- **"KeyError" / `OPENAI_API_KEY` er `None`**
  - sjekk at `.env` ligger i riktig mappe
  - sjekk at du faktisk kjørte `%dotenv` eller `load_dotenv()`

- **Feil kernel** (pakker “finnes ikke”)
  - sjekk `sys.executable` (cellen over)
  - bytt kernel eller installer `ipykernel` og registrer miljøet

- **Du printet nøkkelen i notebook-output**
  - fjern output (clear output) før du pusher til GitHub
  - roter nøkkelen hvis du er usikker på om den er eksponert
